In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score
import datetime

# Load Fashion MNIST dataset
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [2]:
# # Scale the pixel values to be between 0 and 1 and reshape
x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255.0
x_test = x_test.reshape(-1, 28, 28, 1).astype("float32") / 255.0
# Split data into 90% training and 10% validation sets
split = int(0.9 * len(x_train))
x_train, x_valid = x_train[:split], x_train[split:]
y_train, y_valid = y_train[:split], y_train[split:]

In [3]:
# Define Custom model architecture
model = keras.Sequential(
    [
        layers.Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=(28, 28, 1)),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dense(10, activation="softmax"),
    ]
)

# Compile the model
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [4]:
# Define tensorboard logs directory
log_dir = "C:/Users/kstre_3ikvnbf/CS 4375/Project/FashionCustom"
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [5]:
# Train the model
model.fit(x_train, y_train, batch_size=32, epochs=30, validation_data=(x_valid, y_valid), callbacks=[tensorboard_callback])

Epoch 1/30
1688/1688 [==============================] - 70s 40ms/step - loss: 0.4556 - accuracy: 0.8320 - val_loss: 0.3343 - val_accuracy: 0.8797
Epoch 2/30
1688/1688 [==============================] - 105s 62ms/step - loss: 0.3085 - accuracy: 0.8869 - val_loss: 0.3000 - val_accuracy: 0.8865
Epoch 3/30
1688/1688 [==============================] - 105s 62ms/step - loss: 0.2585 - accuracy: 0.9047 - val_loss: 0.2833 - val_accuracy: 0.8972
Epoch 4/30
1688/1688 [==============================] - 109s 64ms/step - loss: 0.2270 - accuracy: 0.9164 - val_loss: 0.2719 - val_accuracy: 0.9027
Epoch 5/30
1688/1688 [==============================] - 108s 64ms/step - loss: 0.2004 - accuracy: 0.9245 - val_loss: 0.2467 - val_accuracy: 0.9070
Epoch 6/30
1688/1688 [==============================] - 112s 66ms/step - loss: 0.1785 - accuracy: 0.9336 - val_loss: 0.2481 - val_accuracy: 0.9103
Epoch 7/30
1688/1688 [==============================] - 112s 66ms/step - loss: 0.1554 - accuracy: 0.9420 - val_loss: 0.

In [6]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(x_test, y_test)

# Generate predictions on the test set
y_pred = model.predict(x_test)

# Convert predicted probabilities to predicted class labels
y_pred_classes = np.argmax(y_pred, axis=1)

313/313 [==============================] - 9s 23ms/step


In [7]:
# Compute evaluation metrics
conf_matrix = confusion_matrix(y_test, y_pred_classes)
precision = precision_score(y_test, y_pred_classes, average='macro')
recall = recall_score(y_test, y_pred_classes, average='macro')
f1score = f1_score(y_test, y_pred_classes, average='macro')
auc_roc = roc_auc_score(y_test, y_pred, multi_class='ovr')

# Print the evaluation metrics
print("Test Loss:", test_loss)
print("Accuracy:", test_acc)
print("Confusion Matrix:\n", conf_matrix)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1score)
print("AU-ROC:", auc_roc)

Test Loss: 0.6791559457778931
Accuracy: 0.9039999842643738
Confusion Matrix:
 [[875   0  14  17   3   2  85   0   3   1]
 [  2 981   0   7   4   0   4   0   2   0]
 [ 20   0 841   5  63   0  71   0   0   0]
 [ 17   9  20 851  57   0  41   0   4   1]
 [  3   0  36  15 885   0  59   0   2   0]
 [  0   0   0   0   0 972   0  13   0  15]
 [132   2  56  21  60   0 728   0   1   0]
 [  0   0   0   0   0  16   0 967   0  17]
 [  6   1   2   1   4   3   9   5 969   0]
 [  1   0   0   0   1   2   0  25   0 971]]
Precision: 0.9052609615989275
Recall: 0.9039999999999999
F1-score: 0.9042480987688271
AU-ROC: 0.9928051888888889


In [8]:
# Log evaluation metrics to TensorBoard
test_writer = tf.summary.create_file_writer(log_dir)
with test_writer.as_default():
    tf.summary.scalar('Accuracy', test_acc, step=0)
    tf.summary.scalar('Precision', precision, step=0)
    tf.summary.scalar('Recall', recall, step=0)
    tf.summary.scalar('F1-score', f1score, step=0)
    tf.summary.scalar('AU-ROC', auc_roc, step=0)

In [9]:
# tensorboard --logdir_spec=FashionMLP:"C:/Users/kstre_3ikvnbf/CS 4375/Project/FashionMLP",FashionLeNet:"C:/Users/kstre_3ikvnbf/CS 4375/Project/FashionLeNet",FashionCustom:"C:/Users/kstre_3ikvnbf/CS 4375/Project/FashionCustom",MnistLeNet:"C:/Users/kstre_3ikvnbf/CS 4375/Project/MnistLeNet",MnistMLP:"C:/Users/kstre_3ikvnbf/CS 4375/Project/MnistMLP",MnistCustom:"C:/Users/kstre_3ikvnbf/CS 4375/Project/MnistCustom"